### References:
    https://pytorch.org/tutorials/beginner/blitz/cifar10_tutorial.html?highlight=cifar
    https://github.com/kuangliu/pytorch-cifar/blob/master/main.py

In [2]:
import torch
import torchvision

In [131]:
import torch.nn as nn
import torch.nn.functional as F

class BasicBlock(nn.Module):
    expansion = 1

    def __init__(self, in_planes, planes, stride=1):
        super(BasicBlock, self).__init__()
        self.conv1 = nn.Conv2d(
            in_planes, planes, kernel_size=3, stride=stride, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(planes)
        self.conv2 = nn.Conv2d(planes, planes, kernel_size=3,
                               stride=1, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(planes)

        self.shortcut = nn.Sequential()
        if stride != 1 or in_planes != self.expansion*planes:
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_planes, self.expansion*planes,
                          kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(self.expansion*planes)
            )

    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = self.bn2(self.conv2(out))
        out += self.shortcut(x)
        out = F.relu(out)
        return out
    
class ResNet(nn.Module):
    def __init__(self, block, num_blocks, num_classes=10):
        super(ResNet, self).__init__()
        self.in_planes = 64

        self.conv1 = nn.Conv2d(3, 64, kernel_size=3,
                               stride=1, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(64)
        self.layer1 = self._make_layer(block, 64, num_blocks[0], stride=1)
        self.layer2 = self._make_layer(block, 128, num_blocks[1], stride=2)
        self.layer3 = self._make_layer(block, 256, num_blocks[2], stride=2)
        self.layer4 = self._make_layer(block, 512, num_blocks[2], stride=2)
        self.linear = nn.Linear(512*block.expansion, num_classes)

    def _make_layer(self, block, planes, num_blocks, stride):
        strides = [stride] + [1]*(num_blocks-1)
        layers = []
        for stride in strides:
            layers.append(block(self.in_planes, planes, stride))
            self.in_planes = planes * block.expansion
        return nn.Sequential(*layers)

    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = self.layer1(out)
        out = self.layer2(out)
        out = self.layer3(out)
        out = self.layer4(out)
        out = F.avg_pool2d(out, 4)
        out = out.view(out.size(0), -1)
        out = self.linear(out)
        return out

In [132]:
model = ResNet(BasicBlock, [1, 1, 1, 1])
sum(p.numel() for p in model.parameters() if p.requires_grad)

4903242

In [118]:
GPU_BATCH_SIZE_LIMIT = 512

def train(model, criterion, optimizer, dataloader, device, scheduler = None, use_scheduler = False, batch_size = 32):
    model.train()
    train_loss = 0
    correct = 0
    count = 0
    total = 0
    for i, data in enumerate(dataloader, 0):
        image, label = data
        image = image.to(device)
        label = label.to(device)
        
        if batch_size >= GPU_BATCH_SIZE_LIMIT:
            if(count == 0):
                optimizer.step()
                optimizer.zero_grad()
                count = batch_size/32

            output = model(image)

            loss = criterion(output, label)*32/batch_size
            loss.backward()
            count = count - 1

            pred = torch.max(output.data, 1)[1]
            
            cur_correct = (pred == label).sum().item()
            cur_loss = loss.item()
        
        else:
            optimizer.zero_grad()
            output = model(image)
            loss = criterion(output, label)

            train_loss += loss.item()

            pred = torch.max(output.data, 1)[1]
            cur_correct = (pred == label).sum().item()
            cur_loss = loss.item()

            loss.backward()
            optimizer.step()
        
        total += label.size(0)
        correct += cur_correct
        train_loss += cur_loss

        if use_scheduler:
            scheduler.step()

    accuracy = correct/total
    train_loss = train_loss/len(dataloader)

    return train_loss, accuracy

def test(model, criterion, dataloader, device, batch_size = 32):
    model.eval()
    test_loss = 0
    correct = 0
    count = 0
    total = 0
    for i, data in enumerate(dataloader, 0):
        image, label = data
        image = image.to(device)
        label = label.to(device)
                
        if batch_size >= GPU_BATCH_SIZE_LIMIT:
            if(count == 0):
                count = batch_size/32

            output = model(image)

            loss = criterion(output, label)*32/batch_size
            count = count - 1

            pred = torch.max(output.data, 1)[1]
            
            cur_correct = (pred == label).sum().item()
            cur_loss = loss.item()
        
        else:
            output = model(image)
            loss = criterion(output, label)

            pred = torch.max(output.data, 1)[1]
            cur_correct = (pred == label).sum().item()
            cur_loss = loss.item()
            
        total += label.size(0)
        correct += cur_correct
        test_loss += cur_loss

    accuracy = correct/total
    test_loss = test_loss/len(dataloader)

    return test_loss, accuracy

In [119]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

### Experiment 1

In [15]:
batch_size = 32

transform = torchvision.transforms.Compose(
    [torchvision.transforms.ToTensor(), torchvision.transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010))])

trainset = torchvision.datasets.CIFAR10(root = './data', train = True, download = True, transform = transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size = batch_size, shuffle = True)

testset = torchvision.datasets.CIFAR10(root = './data', train = False, download = True, transform = transform)
testloader = torch.utils.data.DataLoader(testset, batch_size = batch_size, shuffle = False)

classes = ('plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

model = ResNet(BasicBlock, [2, 2, 2, 2])
model.to(device)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.001, momentum=0.9, weight_decay=0.0001)


train_losses_ = []
train_accuracies_ = []
test_losses_ = []
test_accuracies_ = []

epochs = 5
for epoch in range(epochs):
    print(f"\n\tEpoch: {epoch}")
    
    train_loss, train_accuracy = train(model, criterion, optimizer, trainloader, device)
    train_losses_.append(train_loss)
    train_accuracies_.append(train_accuracy)
    print(f"\tTraining Loss: {round(train_loss, 4)}; Training Accuracy: {round(train_accuracy*100, 4)}%")
    
    test_loss, test_accuracy = test(model, criterion, testloader, device)
    test_losses_.append(test_loss)
    test_accuracies_.append(test_accuracy)
    print(f"\tTesting Loss: {round(test_loss, 4)}; Testing Accuracy: {round(test_accuracy*100, 4)}%")


	Epoch: 0
	Training Loss: 2.5645; Training Accuracy: 53.516%
	Testing Loss: 0.9553; Testing Accuracy: 65.75%

	Epoch: 1
	Training Loss: 1.5774; Training Accuracy: 72.306%
	Testing Loss: 0.7048; Testing Accuracy: 75.13%

	Epoch: 2
	Training Loss: 1.1238; Training Accuracy: 80.514%
	Testing Loss: 0.6602; Testing Accuracy: 77.31%

	Epoch: 3
	Training Loss: 0.7937; Training Accuracy: 86.16%
	Testing Loss: 0.7947; Testing Accuracy: 75.13%

	Epoch: 4
	Training Loss: 0.5269; Training Accuracy: 91.118%
	Testing Loss: 0.7802; Testing Accuracy: 75.85%


### Experiment 2

In [17]:
batch_size = 32

transform_train = torchvision.transforms.Compose([
                    torchvision.transforms.RandomCrop(32, padding=4),
                    torchvision.transforms.RandomHorizontalFlip(),
                    torchvision.transforms.ToTensor(),
                    torchvision.transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])

transform_test = torchvision.transforms.Compose([
                    torchvision.transforms.ToTensor(), 
                    torchvision.transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010))])

trainset = torchvision.datasets.CIFAR10(root = './data', train = True, download = True, transform = transform_train)
trainloader = torch.utils.data.DataLoader(trainset, batch_size = batch_size, shuffle = True)

testset = torchvision.datasets.CIFAR10(root = './data', train = False, download = True, transform = transform_test)
testloader = torch.utils.data.DataLoader(testset, batch_size = batch_size, shuffle = False)

classes = ('plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

model = ResNet(BasicBlock, [2, 2, 2, 2])
model.to(device)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.001, momentum=0.9, weight_decay=0.0001)


train_losses_ = []
train_accuracies_ = []
test_losses_ = []
test_accuracies_ = []

epochs = 5
for epoch in range(epochs):
    print(f"\n\tEpoch: {epoch}")
    
    train_loss, train_accuracy = train(model, criterion, optimizer, trainloader, device)
    train_losses_.append(train_loss)
    train_accuracies_.append(train_accuracy)
    print(f"\tTraining Loss: {round(train_loss, 4)}; Training Accuracy: {round(train_accuracy*100, 4)}%")
    
    test_loss, test_accuracy = test(model, criterion, testloader, device)
    test_losses_.append(test_loss)
    test_accuracies_.append(test_accuracy)
    print(f"\tTesting Loss: {round(test_loss, 4)}; Testing Accuracy: {round(test_accuracy*100, 4)}%")

Files already downloaded and verified
Files already downloaded and verified

	Epoch: 0
	Training Loss: 2.8569; Training Accuracy: 47.698%
	Testing Loss: 1.2013; Testing Accuracy: 57.86%

	Epoch: 1
	Training Loss: 1.9058; Training Accuracy: 66.156%
	Testing Loss: 0.8083; Testing Accuracy: 71.94%

	Epoch: 2
	Training Loss: 1.5045; Training Accuracy: 73.854%
	Testing Loss: 0.7316; Testing Accuracy: 75.75%

	Epoch: 3
	Training Loss: 1.274; Training Accuracy: 77.948%
	Testing Loss: 0.5878; Testing Accuracy: 80.14%

	Epoch: 4
	Training Loss: 1.1059; Training Accuracy: 80.852%
	Testing Loss: 0.577; Testing Accuracy: 80.62%


### Experiment 3

In [133]:
batch_size = 32

transform_train = torchvision.transforms.Compose([
                    torchvision.transforms.RandomCrop(32, padding=4),
                    torchvision.transforms.RandomHorizontalFlip(),
                    torchvision.transforms.ToTensor(),
                    torchvision.transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])

transform_test = torchvision.transforms.Compose([
                    torchvision.transforms.ToTensor(), 
                    torchvision.transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010))])

trainset = torchvision.datasets.CIFAR10(root = './data', train = True, download = True, transform = transform_train)
trainloader = torch.utils.data.DataLoader(trainset, batch_size = batch_size, shuffle = True)

testset = torchvision.datasets.CIFAR10(root = './data', train = False, download = True, transform = transform_test)
testloader = torch.utils.data.DataLoader(testset, batch_size = batch_size, shuffle = False)

classes = ('plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

model = ResNet(BasicBlock, [2, 2, 2, 2])
model.to(device)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.01, momentum=0.9, weight_decay=0.0001)


train_losses_ = []
train_accuracies_ = []
test_losses_ = []
test_accuracies_ = []

epochs = 5
for epoch in range(epochs):
    print(f"\n\tEpoch: {epoch}")
    
    train_loss, train_accuracy = train(model, criterion, optimizer, trainloader, device)
    train_losses_.append(train_loss)
    train_accuracies_.append(train_accuracy)
    print(f"\tTraining Loss: {round(train_loss, 4)}; Training Accuracy: {round(train_accuracy*100, 4)}%")
    
    test_loss, test_accuracy = test(model, criterion, testloader, device)
    test_losses_.append(test_loss)
    test_accuracies_.append(test_accuracy)
    print(f"\tTesting Loss: {round(test_loss, 4)}; Testing Accuracy: {round(test_accuracy*100, 4)}%")

Files already downloaded and verified
Files already downloaded and verified

	Epoch: 0
	Training Loss: 3.07; Training Accuracy: 43.998%
	Testing Loss: 1.2807; Testing Accuracy: 54.54%

	Epoch: 1
	Training Loss: 1.9856; Training Accuracy: 64.784%
	Testing Loss: 0.8322; Testing Accuracy: 71.09%

	Epoch: 2
	Training Loss: 1.5254; Training Accuracy: 73.45%
	Testing Loss: 0.6997; Testing Accuracy: 75.74%

	Epoch: 3
	Training Loss: 1.2786; Training Accuracy: 77.93%
	Testing Loss: 0.6192; Testing Accuracy: 79.22%

	Epoch: 4
	Training Loss: 1.1156; Training Accuracy: 80.71%
	Testing Loss: 0.5351; Testing Accuracy: 82.01%


### Experiment 4

In [20]:
batch_size = 32

transform_train = torchvision.transforms.Compose([
                    torchvision.transforms.RandomCrop(32, padding=4),
                    torchvision.transforms.RandomHorizontalFlip(),
                    torchvision.transforms.ToTensor(),
                    torchvision.transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])

transform_test = torchvision.transforms.Compose([
                    torchvision.transforms.ToTensor(), 
                    torchvision.transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010))])

trainset = torchvision.datasets.CIFAR10(root = './data', train = True, download = True, transform = transform_train)
trainloader = torch.utils.data.DataLoader(trainset, batch_size = batch_size, shuffle = True)

testset = torchvision.datasets.CIFAR10(root = './data', train = False, download = True, transform = transform_test)
testloader = torch.utils.data.DataLoader(testset, batch_size = batch_size, shuffle = False)

classes = ('plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

model = ResNet(BasicBlock, [2, 2, 2, 2])
model.to(device)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.1, momentum=0.9, weight_decay=0.0001)


train_losses_ = []
train_accuracies_ = []
test_losses_ = []
test_accuracies_ = []

epochs = 5
for epoch in range(epochs):
    print(f"\n\tEpoch: {epoch}")
    
    train_loss, train_accuracy = train(model, criterion, optimizer, trainloader, device)
    train_losses_.append(train_loss)
    train_accuracies_.append(train_accuracy)
    print(f"\tTraining Loss: {round(train_loss, 4)}; Training Accuracy: {round(train_accuracy*100, 4)}%")
    
    test_loss, test_accuracy = test(model, criterion, testloader, device)
    test_losses_.append(test_loss)
    test_accuracies_.append(test_accuracy)
    print(f"\tTesting Loss: {round(test_loss, 4)}; Testing Accuracy: {round(test_accuracy*100, 4)}%")

Files already downloaded and verified
Files already downloaded and verified

	Epoch: 0
	Training Loss: 4.0013; Training Accuracy: 27.402%
	Testing Loss: 1.6766; Testing Accuracy: 35.77%

	Epoch: 1
	Training Loss: 3.0418; Training Accuracy: 44.444%
	Testing Loss: 1.265; Testing Accuracy: 54.06%

	Epoch: 2
	Training Loss: 2.4568; Training Accuracy: 56.274%
	Testing Loss: 1.1245; Testing Accuracy: 61.08%

	Epoch: 3
	Training Loss: 2.0156; Training Accuracy: 64.484%
	Testing Loss: 0.9501; Testing Accuracy: 67.33%

	Epoch: 4
	Training Loss: 1.6487; Training Accuracy: 71.168%
	Testing Loss: 0.7452; Testing Accuracy: 73.77%


### Experiment 5

In [21]:
batch_size = 32

transform_train = torchvision.transforms.Compose([
                    torchvision.transforms.RandomCrop(32, padding=4),
                    torchvision.transforms.RandomHorizontalFlip(),
                    torchvision.transforms.ToTensor(),
                    torchvision.transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])

transform_test = torchvision.transforms.Compose([
                    torchvision.transforms.ToTensor(), 
                    torchvision.transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010))])

trainset = torchvision.datasets.CIFAR10(root = './data', train = True, download = True, transform = transform_train)
trainloader = torch.utils.data.DataLoader(trainset, batch_size = batch_size, shuffle = True)

testset = torchvision.datasets.CIFAR10(root = './data', train = False, download = True, transform = transform_test)
testloader = torch.utils.data.DataLoader(testset, batch_size = batch_size, shuffle = False)

classes = ('plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

model = ResNet(BasicBlock, [2, 2, 2, 2])
model.to(device)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.01, weight_decay=0.0001)


train_losses_ = []
train_accuracies_ = []
test_losses_ = []
test_accuracies_ = []

epochs = 5
for epoch in range(epochs):
    print(f"\n\tEpoch: {epoch}")
    
    train_loss, train_accuracy = train(model, criterion, optimizer, trainloader, device)
    train_losses_.append(train_loss)
    train_accuracies_.append(train_accuracy)
    print(f"\tTraining Loss: {round(train_loss, 4)}; Training Accuracy: {round(train_accuracy*100, 4)}%")
    
    test_loss, test_accuracy = test(model, criterion, testloader, device)
    test_losses_.append(test_loss)
    test_accuracies_.append(test_accuracy)
    print(f"\tTesting Loss: {round(test_loss, 4)}; Testing Accuracy: {round(test_accuracy*100, 4)}%")

Files already downloaded and verified
Files already downloaded and verified

	Epoch: 0
	Training Loss: 3.7236; Training Accuracy: 30.386%
	Testing Loss: 1.6455; Testing Accuracy: 39.45%

	Epoch: 1
	Training Loss: 2.8793; Training Accuracy: 46.78%
	Testing Loss: 1.3395; Testing Accuracy: 52.06%

	Epoch: 2
	Training Loss: 2.4247; Training Accuracy: 56.208%
	Testing Loss: 1.24; Testing Accuracy: 57.35%

	Epoch: 3
	Training Loss: 2.221; Training Accuracy: 60.128%
	Testing Loss: 1.288; Testing Accuracy: 55.65%

	Epoch: 4
	Training Loss: 2.1067; Training Accuracy: 62.324%
	Testing Loss: 1.1168; Testing Accuracy: 60.53%


### Experiment 6

In [22]:
batch_size = 32

transform_train = torchvision.transforms.Compose([
                    torchvision.transforms.RandomCrop(32, padding=4),
                    torchvision.transforms.RandomHorizontalFlip(),
                    torchvision.transforms.ToTensor(),
                    torchvision.transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])

transform_test = torchvision.transforms.Compose([
                    torchvision.transforms.ToTensor(), 
                    torchvision.transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010))])

trainset = torchvision.datasets.CIFAR10(root = './data', train = True, download = True, transform = transform_train)
trainloader = torch.utils.data.DataLoader(trainset, batch_size = batch_size, shuffle = True)

testset = torchvision.datasets.CIFAR10(root = './data', train = False, download = True, transform = transform_test)
testloader = torch.utils.data.DataLoader(testset, batch_size = batch_size, shuffle = False)

classes = ('plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

model = ResNet(BasicBlock, [2, 2, 2, 2])
model.to(device)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adagrad(model.parameters(), lr=0.01, weight_decay=0.0001)


train_losses_ = []
train_accuracies_ = []
test_losses_ = []
test_accuracies_ = []

epochs = 5
for epoch in range(epochs):
    print(f"\n\tEpoch: {epoch}")
    
    train_loss, train_accuracy = train(model, criterion, optimizer, trainloader, device)
    train_losses_.append(train_loss)
    train_accuracies_.append(train_accuracy)
    print(f"\tTraining Loss: {round(train_loss, 4)}; Training Accuracy: {round(train_accuracy*100, 4)}%")
    
    test_loss, test_accuracy = test(model, criterion, testloader, device)
    test_losses_.append(test_loss)
    test_accuracies_.append(test_accuracy)
    print(f"\tTesting Loss: {round(test_loss, 4)}; Testing Accuracy: {round(test_accuracy*100, 4)}%")

Files already downloaded and verified
Files already downloaded and verified

	Epoch: 0
	Training Loss: 3.1805; Training Accuracy: 41.224%
	Testing Loss: 1.259; Testing Accuracy: 53.73%

	Epoch: 1
	Training Loss: 2.128; Training Accuracy: 62.156%
	Testing Loss: 0.913; Testing Accuracy: 67.99%

	Epoch: 2
	Training Loss: 1.7074; Training Accuracy: 69.984%
	Testing Loss: 0.854; Testing Accuracy: 69.78%

	Epoch: 3
	Training Loss: 1.4342; Training Accuracy: 74.932%
	Testing Loss: 0.632; Testing Accuracy: 78.26%

	Epoch: 4
	Training Loss: 1.2349; Training Accuracy: 78.432%
	Testing Loss: 0.6437; Testing Accuracy: 78.13%


### Experiment 7

In [23]:
batch_size = 32

transform_train = torchvision.transforms.Compose([
                    torchvision.transforms.RandomCrop(32, padding=4),
                    torchvision.transforms.RandomHorizontalFlip(),
                    torchvision.transforms.ToTensor(),
                    torchvision.transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])

transform_test = torchvision.transforms.Compose([
                    torchvision.transforms.ToTensor(), 
                    torchvision.transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010))])

trainset = torchvision.datasets.CIFAR10(root = './data', train = True, download = True, transform = transform_train)
trainloader = torch.utils.data.DataLoader(trainset, batch_size = batch_size, shuffle = True)

testset = torchvision.datasets.CIFAR10(root = './data', train = False, download = True, transform = transform_test)
testloader = torch.utils.data.DataLoader(testset, batch_size = batch_size, shuffle = False)

classes = ('plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

model = ResNet(BasicBlock, [2, 2, 2, 2])
model.to(device)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adadelta(model.parameters(), lr=0.01, weight_decay=0.0001)


train_losses_ = []
train_accuracies_ = []
test_losses_ = []
test_accuracies_ = []

epochs = 5
for epoch in range(epochs):
    print(f"\n\tEpoch: {epoch}")
    
    train_loss, train_accuracy = train(model, criterion, optimizer, trainloader, device)
    train_losses_.append(train_loss)
    train_accuracies_.append(train_accuracy)
    print(f"\tTraining Loss: {round(train_loss, 4)}; Training Accuracy: {round(train_accuracy*100, 4)}%")
    
    test_loss, test_accuracy = test(model, criterion, testloader, device)
    test_losses_.append(test_loss)
    test_accuracies_.append(test_accuracy)
    print(f"\tTesting Loss: {round(test_loss, 4)}; Testing Accuracy: {round(test_accuracy*100, 4)}%")

Files already downloaded and verified
Files already downloaded and verified

	Epoch: 0
	Training Loss: 3.0538; Training Accuracy: 43.89%
	Testing Loss: 1.23; Testing Accuracy: 55.69%

	Epoch: 1
	Training Loss: 2.2977; Training Accuracy: 58.824%
	Testing Loss: 1.0106; Testing Accuracy: 63.25%

	Epoch: 2
	Training Loss: 1.9435; Training Accuracy: 65.47%
	Testing Loss: 0.9031; Testing Accuracy: 68.61%

	Epoch: 3
	Training Loss: 1.6834; Training Accuracy: 70.346%
	Testing Loss: 0.7526; Testing Accuracy: 73.84%

	Epoch: 4
	Training Loss: 1.4895; Training Accuracy: 73.938%
	Testing Loss: 0.8041; Testing Accuracy: 73.3%


### Experiment 8

In [26]:
batch_size = 64

transform_train = torchvision.transforms.Compose([
                    torchvision.transforms.RandomCrop(32, padding=4),
                    torchvision.transforms.RandomHorizontalFlip(),
                    torchvision.transforms.ToTensor(),
                    torchvision.transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])

transform_test = torchvision.transforms.Compose([
                    torchvision.transforms.ToTensor(), 
                    torchvision.transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010))])

trainset = torchvision.datasets.CIFAR10(root = './data', train = True, download = True, transform = transform_train)
trainloader = torch.utils.data.DataLoader(trainset, batch_size = batch_size, shuffle = True)

testset = torchvision.datasets.CIFAR10(root = './data', train = False, download = True, transform = transform_test)
testloader = torch.utils.data.DataLoader(testset, batch_size = batch_size, shuffle = False)

classes = ('plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

model = ResNet(BasicBlock, [2, 2, 2, 2])
model.to(device)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.01, momentum=0.9, weight_decay=0.0001)


train_losses_ = []
train_accuracies_ = []
test_losses_ = []
test_accuracies_ = []

epochs = 5
for epoch in range(epochs):
    print(f"\n\tEpoch: {epoch}")
    
    train_loss, train_accuracy = train(model, criterion, optimizer, trainloader, device, batch_size = 64)
    train_losses_.append(train_loss)
    train_accuracies_.append(train_accuracy)
    print(f"\tTraining Loss: {round(train_loss, 4)}; Training Accuracy: {round(train_accuracy*100, 4)}%")
    
    test_loss, test_accuracy = test(model, criterion, testloader, device, batch_size = 64)
    test_losses_.append(test_loss)
    test_accuracies_.append(test_accuracy)
    print(f"\tTesting Loss: {round(test_loss, 4)}; Testing Accuracy: {round(test_accuracy*100, 4)}%")

Files already downloaded and verified
Files already downloaded and verified

	Epoch: 0
	Training Loss: 2.9855; Training Accuracy: 45.576%
	Testing Loss: 1.3195; Testing Accuracy: 55.37%

	Epoch: 1
	Training Loss: 1.9414; Training Accuracy: 65.68%
	Testing Loss: 0.9677; Testing Accuracy: 67.38%

	Epoch: 2
	Training Loss: 1.4932; Training Accuracy: 74.058%
	Testing Loss: 0.8003; Testing Accuracy: 73.01%

	Epoch: 3
	Training Loss: 1.2477; Training Accuracy: 78.316%
	Testing Loss: 0.6142; Testing Accuracy: 79.1%

	Epoch: 4
	Training Loss: 1.083; Training Accuracy: 81.192%
	Testing Loss: 0.6038; Testing Accuracy: 80.0%


### Experiment 9

In [27]:
batch_size = 128

transform_train = torchvision.transforms.Compose([
                    torchvision.transforms.RandomCrop(32, padding=4),
                    torchvision.transforms.RandomHorizontalFlip(),
                    torchvision.transforms.ToTensor(),
                    torchvision.transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])

transform_test = torchvision.transforms.Compose([
                    torchvision.transforms.ToTensor(), 
                    torchvision.transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010))])

trainset = torchvision.datasets.CIFAR10(root = './data', train = True, download = True, transform = transform_train)
trainloader = torch.utils.data.DataLoader(trainset, batch_size = batch_size, shuffle = True)

testset = torchvision.datasets.CIFAR10(root = './data', train = False, download = True, transform = transform_test)
testloader = torch.utils.data.DataLoader(testset, batch_size = batch_size, shuffle = False)

classes = ('plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

model = ResNet(BasicBlock, [2, 2, 2, 2])
model.to(device)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.01, momentum=0.9, weight_decay=0.0001)


train_losses_ = []
train_accuracies_ = []
test_losses_ = []
test_accuracies_ = []

epochs = 5
for epoch in range(epochs):
    print(f"\n\tEpoch: {epoch}")
    
    train_loss, train_accuracy = train(model, criterion, optimizer, trainloader, device, batch_size = 128)
    train_losses_.append(train_loss)
    train_accuracies_.append(train_accuracy)
    print(f"\tTraining Loss: {round(train_loss, 4)}; Training Accuracy: {round(train_accuracy*100, 4)}%")
    
    test_loss, test_accuracy = test(model, criterion, testloader, device, batch_size = 128)
    test_losses_.append(test_loss)
    test_accuracies_.append(test_accuracy)
    print(f"\tTesting Loss: {round(test_loss, 4)}; Testing Accuracy: {round(test_accuracy*100, 4)}%")

Files already downloaded and verified
Files already downloaded and verified

	Epoch: 0
	Training Loss: 2.886; Training Accuracy: 47.34%
	Testing Loss: 1.2649; Testing Accuracy: 55.76%

	Epoch: 1
	Training Loss: 1.886; Training Accuracy: 66.504%
	Testing Loss: 0.9538; Testing Accuracy: 69.04%

	Epoch: 2
	Training Loss: 1.4315; Training Accuracy: 74.934%
	Testing Loss: 0.6605; Testing Accuracy: 77.32%

	Epoch: 3
	Training Loss: 1.19; Training Accuracy: 79.278%
	Testing Loss: 0.6034; Testing Accuracy: 79.86%

	Epoch: 4
	Training Loss: 1.0344; Training Accuracy: 82.042%
	Testing Loss: 0.6107; Testing Accuracy: 80.27%


### Experiment 10

In [28]:
batch_size = 256

transform_train = torchvision.transforms.Compose([
                    torchvision.transforms.RandomCrop(32, padding=4),
                    torchvision.transforms.RandomHorizontalFlip(),
                    torchvision.transforms.ToTensor(),
                    torchvision.transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])

transform_test = torchvision.transforms.Compose([
                    torchvision.transforms.ToTensor(), 
                    torchvision.transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010))])

trainset = torchvision.datasets.CIFAR10(root = './data', train = True, download = True, transform = transform_train)
trainloader = torch.utils.data.DataLoader(trainset, batch_size = batch_size, shuffle = True)

testset = torchvision.datasets.CIFAR10(root = './data', train = False, download = True, transform = transform_test)
testloader = torch.utils.data.DataLoader(testset, batch_size = batch_size, shuffle = False)

classes = ('plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

model = ResNet(BasicBlock, [2, 2, 2, 2])
model.to(device)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.01, momentum=0.9, weight_decay=0.0001)


train_losses_ = []
train_accuracies_ = []
test_losses_ = []
test_accuracies_ = []

epochs = 5
for epoch in range(epochs):
    print(f"\n\tEpoch: {epoch}")
    
    train_loss, train_accuracy = train(model, criterion, optimizer, trainloader, device, batch_size = 256)
    train_losses_.append(train_loss)
    train_accuracies_.append(train_accuracy)
    print(f"\tTraining Loss: {round(train_loss, 4)}; Training Accuracy: {round(train_accuracy*100, 4)}%")
    
    test_loss, test_accuracy = test(model, criterion, testloader, device, batch_size = 256)
    test_losses_.append(test_loss)
    test_accuracies_.append(test_accuracy)
    print(f"\tTesting Loss: {round(test_loss, 4)}; Testing Accuracy: {round(test_accuracy*100, 4)}%")

Files already downloaded and verified
Files already downloaded and verified

	Epoch: 0
	Training Loss: 3.0066; Training Accuracy: 44.648%
	Testing Loss: 1.3332; Testing Accuracy: 52.62%

	Epoch: 1
	Training Loss: 2.0033; Training Accuracy: 64.266%
	Testing Loss: 0.9643; Testing Accuracy: 67.22%

	Epoch: 2
	Training Loss: 1.5359; Training Accuracy: 72.85%
	Testing Loss: 0.8235; Testing Accuracy: 72.64%

	Epoch: 3
	Training Loss: 1.2628; Training Accuracy: 77.83%
	Testing Loss: 0.7223; Testing Accuracy: 76.31%

	Epoch: 4
	Training Loss: 1.0916; Training Accuracy: 80.866%
	Testing Loss: 0.6643; Testing Accuracy: 77.08%


In [29]:
torch.cuda.empty_cache()

### Experiment 11

In [ ]:
batch_size = 32

transform_train = torchvision.transforms.Compose([
                    torchvision.transforms.RandomCrop(32, padding=4),
                    torchvision.transforms.RandomHorizontalFlip(),
                    torchvision.transforms.ToTensor(),
                    torchvision.transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])

transform_test = torchvision.transforms.Compose([
                    torchvision.transforms.ToTensor(), 
                    torchvision.transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010))])

trainset = torchvision.datasets.CIFAR10(root = './data', train = True, download = True, transform = transform_train)
trainloader = torch.utils.data.DataLoader(trainset, batch_size = batch_size, shuffle = True)

testset = torchvision.datasets.CIFAR10(root = './data', train = False, download = True, transform = transform_test)
testloader = torch.utils.data.DataLoader(testset, batch_size = batch_size, shuffle = False)

classes = ('plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

model = ResNet(BasicBlock, [2, 2, 2, 2])
model.to(device)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.01, momentum=0.9, weight_decay=0.0001)


train_losses_ = []
train_accuracies_ = []
test_losses_ = []
test_accuracies_ = []

epochs = 25
for epoch in range(epochs):
    print(f"\n\tEpoch: {epoch}")
    
    train_loss, train_accuracy = train(model, criterion, optimizer, trainloader, device, batch_size = 32)
    train_losses_.append(train_loss)
    train_accuracies_.append(train_accuracy)
    print(f"\tTraining Loss: {round(train_loss, 4)}; Training Accuracy: {round(train_accuracy*100, 4)}%")
    
    test_loss, test_accuracy = test(model, criterion, testloader, device, batch_size = 32)
    test_losses_.append(test_loss)
    test_accuracies_.append(test_accuracy)
    print(f"\tTesting Loss: {round(test_loss, 4)}; Testing Accuracy: {round(test_accuracy*100, 4)}%")

Files already downloaded and verified
Files already downloaded and verified

	Epoch: 0
	Training Loss: 3.0979; Training Accuracy: 43.474%
	Testing Loss: 1.6336; Testing Accuracy: 48.3%

	Epoch: 1
	Training Loss: 1.9974; Training Accuracy: 64.442%
	Testing Loss: 0.7966; Testing Accuracy: 71.92%

	Epoch: 2
	Training Loss: 1.5179; Training Accuracy: 73.536%
	Testing Loss: 0.7326; Testing Accuracy: 75.39%

	Epoch: 3
	Training Loss: 1.2711; Training Accuracy: 78.002%
	Testing Loss: 0.6211; Testing Accuracy: 79.11%

	Epoch: 4
	Training Loss: 1.107; Training Accuracy: 80.898%
	Testing Loss: 0.519; Testing Accuracy: 82.51%

	Epoch: 5
	Training Loss: 0.9796; Training Accuracy: 83.05%
	Testing Loss: 0.5409; Testing Accuracy: 82.09%

	Epoch: 6
	Training Loss: 0.887; Training Accuracy: 84.758%
	Testing Loss: 0.4434; Testing Accuracy: 84.98%

	Epoch: 7


In [31]:
sum(p.numel() for p in model.parameters() if p.requires_grad)

11173962